In [2]:
from datasets import load_dataset, load_metric, Audio
import numpy as np
import pandas as pd
minds = load_dataset("PolyAI/minds14", name="en-US", split="train")
print(minds)

Reusing dataset minds14 (/Users/maiduydung/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/aa40414f15e0f919231d617440192034af844835dc1e6a697f4b552e0551fd26)


Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 563
})


In [3]:
minds = minds.train_test_split(test_size=0.2)

In [4]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [5]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 113
    })
})

In [6]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/Users/maiduydung/opt/anaconda3/envs/huggingface_2/lib/python3.8/site-packages/transformers/configuration_utils.py:358: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='facebook/wav2vec2-base', vocab_size=32, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

In [8]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))


In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    return df

In [10]:
temp_df = show_random_elements(minds['train'].remove_columns(['path', 'audio']))


,transcription
0,yes hello I'd like to change my address please
1,I lost my card and need to freeze my account
2,I need to make a payment
3,hello I like to know how I be able to deposit the money that I currently have
4,how do I set up a joint account


In [11]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [12]:
vocabs = minds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=minds.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
vocabs_list = list(set(vocabs['train']['vocab'][0]) | set(vocabs['test']['vocab'][0]))
len(vocabs_list)

63

In [14]:
vocab_dict = {v: k for k, v in enumerate(vocabs_list)}
vocab_dict

{'u': 0,
 'k': 1,
 '7': 2,
 'J': 3,
 'w': 4,
 'x': 5,
 'O': 6,
 'l': 7,
 ' ': 8,
 'I': 9,
 'r': 10,
 'd': 11,
 'v': 12,
 'H': 13,
 'E': 14,
 '0': 15,
 'P': 16,
 'V': 17,
 'q': 18,
 'T': 19,
 'Y': 20,
 'c': 21,
 '8': 22,
 '3': 23,
 'B': 24,
 't': 25,
 '$': 26,
 'y': 27,
 '9': 28,
 'M': 29,
 '6': 30,
 'e': 31,
 'K': 32,
 '-': 33,
 'U': 34,
 'm': 35,
 'b': 36,
 'G': 37,
 'p': 38,
 'D': 39,
 'C': 40,
 'i': 41,
 'N': 42,
 '.': 43,
 'S': 44,
 'f': 45,
 'A': 46,
 'F': 47,
 'h': 48,
 '2': 49,
 's': 50,
 '4': 51,
 'n': 52,
 'R': 53,
 "'": 54,
 'W': 55,
 '5': 56,
 'a': 57,
 'g': 58,
 '1': 59,
 'j': 60,
 'z': 61,
 'o': 62}

In [15]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [16]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

65

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [21]:
tokenizer('hello world')

{'input_ids': [48, 31, 7, 7, 62, 8, 4, 62, 10, 7, 11], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#https://colab.research.google.com/drive/1FjTsqbYKphl9kL-eILgUc-bl4zVThL8F?usp=sharing#scrollTo=aQfneNsmlJI0